# Import Required Packages

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.utils.data as data_utils
from torch.nn.modules import MSELoss, L1Loss, BCELoss

import glob
import csv
import cv2
from numpy import array, asarray, ndarray, swapaxes
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
#!pip install torchvision
#!pip install opencv-python

# Download Files

In [3]:
#training controls
batch_size = 64
epochs = 10
training_size = 0.7
learning_rate = 0.0001

# input image dimensions
img_rows, img_cols = 268, 182

In [4]:
# data holders
x_test = []
x_train = []
y_test= []
y_train= []
tempY = []

In [5]:
# opening the dataset
dataset = csv.reader(open("MovieGenre417.csv",encoding="utf8",errors='replace'), delimiter=",")

# skipping the header line
next(dataset)

['', 'index', 'imdbId', 'Imdb Link', 'Title', 'IMDB Score', 'Genre', 'Poster']

In [6]:
# extract images from zip folder

import zipfile as zf

files = zf.ZipFile("FinalProjectUp.zip", 'r')
files.extractall()
files.close()

In [7]:
# list of image files in SampleMoviePosters folder
flist=glob.glob('FinalProject/*.jpg')  

In [8]:
len(flist)

5384

In [9]:
image_ids = []

for path in flist:
    start = path.rfind("/")+1
    end = len(path)-4
    image_ids.append(path[start:end])
    
#image_ids

In [10]:
import pandas as pd

dataset2 = pd.read_csv("MovieGenre417.csv")
dataset2

,Unnamed: 0,index,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,0,0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,2,2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
2,11,11,112896,http://www.imdb.com/title/tt112896,Dracula: Dead and Loving It (1995),5.8,Comedy|Fantasy|Horror,https://images-na.ssl-images-amazon.com/images...
3,23,23,114168,http://www.imdb.com/title/tt114168,Powder (1995),6.5,Drama|Fantasy|Mystery,https://images-na.ssl-images-amazon.com/images...
4,26,26,114011,http://www.imdb.com/title/tt114011,Now and Then (1995),6.8,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
...,...,...,...,...,...,...,...,...
5379,39322,40060,4508542,http://www.imdb.com/title/tt4508542,Nacida para ganar (2016),4.8,Comedy,https://images-na.ssl-images-amazon.com/images...
5380,39333,40071,1018706,http://www.imdb.com/title/tt1018706,CÌ£o Sem Dono (2007),6.9,Drama,https://images-na.ssl-images-amazon.com/images...
5381,39347,40085,4189294,http://www.imdb.com/title/tt4189294,Lego DC Comics: Batman Be-Leaguered (2014),6.8,Animation|Short|Action,https://images-na.ssl-images-amazon.com/images...
5382,39362,40100,98216,http://www.imdb.com/title/tt98216,Roller Blade Warriors: Taken by Force (1989),3.9,Action|Adventure|Sci-Fi,https://images-na.ssl-images-amazon.com/images...


# Data Preprocessing

In [11]:
y = []
indexlist = []
classes = tuple()
ids = dataset2.imdbId.values.tolist()
for image_id in image_ids:
    #print(dataset2["imdbId"])
    genres = tuple((dataset2[dataset2["imdbId"] == int(image_id)]["Genre"].values[0]).split("|"))
    if int(image_id) in ids:
        indexlist.append(image_id)
    y.append(genres)
    classes = classes + genres
mlb = MultiLabelBinarizer()
mlb.fit(y)
y = mlb.transform(y)
classes = set(classes)
classes = list(classes)
classes.sort()

In [12]:
y_df = pd.DataFrame(y, columns = classes, index = indexlist)
y_df = y_df[['Drama', 'Comedy', 'Romance', 'Action', 'Crime']]
classes = y_df.columns.tolist()
y_df

,Drama,Comedy,Romance,Action,Crime
2371315,1,0,0,0,1
2504404,1,0,0,0,1
363473,1,0,0,0,0
389778,1,0,0,0,0
42693,0,1,0,1,0
...,...,...,...,...,...
409904,0,0,0,1,0
106685,1,1,0,0,0
3375370,0,1,0,0,0
3331846,1,1,0,0,0


In [13]:
y_df_reset = y_df.reset_index()

shape = y_df_reset.shape[1]

index_value = []
genre_lst = []

for i in range(len(y_df_reset)):
    index_value.append(int(y_df_reset.loc[i,"index"]))
    temp_list = []
    for j in y_df_reset.columns[1:]:
        temp_list.append(y_df_reset.loc[i,j])
    genre_lst.append(temp_list)

df = pd.DataFrame(list(zip(index_value, genre_lst)),
               columns =['imdbId', 'genrelst'])

result = dataset2.merge(df, on="imdbId")
result

,Unnamed: 0,index,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster,genrelst
0,0,0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...,"[0, 1, 0, 0, 0]"
1,2,2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 1, 1, 0, 0]"
2,11,11,112896,http://www.imdb.com/title/tt112896,Dracula: Dead and Loving It (1995),5.8,Comedy|Fantasy|Horror,https://images-na.ssl-images-amazon.com/images...,"[0, 1, 0, 0, 0]"
3,23,23,114168,http://www.imdb.com/title/tt114168,Powder (1995),6.5,Drama|Fantasy|Mystery,https://images-na.ssl-images-amazon.com/images...,"[1, 0, 0, 0, 0]"
4,26,26,114011,http://www.imdb.com/title/tt114011,Now and Then (1995),6.8,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...,"[1, 1, 1, 0, 0]"
...,...,...,...,...,...,...,...,...,...
5379,39322,40060,4508542,http://www.imdb.com/title/tt4508542,Nacida para ganar (2016),4.8,Comedy,https://images-na.ssl-images-amazon.com/images...,"[0, 1, 0, 0, 0]"
5380,39333,40071,1018706,http://www.imdb.com/title/tt1018706,CÌ£o Sem Dono (2007),6.9,Drama,https://images-na.ssl-images-amazon.com/images...,"[1, 0, 0, 0, 0]"
5381,39347,40085,4189294,http://www.imdb.com/title/tt4189294,Lego DC Comics: Batman Be-Leaguered (2014),6.8,Animation|Short|Action,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 1, 0]"
5382,39362,40100,98216,http://www.imdb.com/title/tt98216,Roller Blade Warriors: Taken by Force (1989),3.9,Action|Adventure|Sci-Fi,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 1, 0]"


In [14]:
for x in range(len(result)):
    tempY.append((int(result['imdbId'].iloc[x]),result['genrelst'].iloc[x]))

#tempY

# Train/Test Split

In [15]:
#setting the length of training data
length=int(len(flist)*training_size)
length

3768

In [16]:
#extracting the data about the images that are available
i=0
for filename in flist:
    name=int(filename.split('/')[-1][:-4])
    for z in tempY:
        if(z[0]==name):
            
            img = array(cv2.imread(filename))
            img = swapaxes(img, 2,0)
            img = swapaxes(img, 2,1)

            if(i<length):
                x_train.append(img)
                y_train.append(z[1])
                i+=1
            else:
                x_test.append(img)
                y_test.append(z[1])
                i+=1

In [17]:
#converting the data from lists to numpy arrays
x_train=asarray(x_train,dtype=float)
x_test=asarray(x_test,dtype=float)
y_train=asarray(y_train,dtype=float)
y_test=asarray(y_test,dtype=float)

In [18]:
#scaling down the RGB data
x_train /= 255
x_test /= 255

In [19]:
#printing stats about the features
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (3768, 3, 268, 182)
3768 train samples
1616 test samples


In [20]:
train_length = x_train.shape[0]

x_train=torch.from_numpy(x_train)
x_test=torch.from_numpy(x_test)
y_train=torch.from_numpy(y_train)
y_test=torch.from_numpy(y_test)

train = data_utils.TensorDataset(x_train, y_train)
train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True)

test = data_utils.TensorDataset(x_test, y_test)
test_loader = data_utils.DataLoader(test, batch_size=batch_size, shuffle=False)

In [21]:
# Metric calculation

def metric(scores, targets):
    """
    :param scores: the output the model predict
    :param targets: the gt label
    :return: OP, OR, OF1, CP, CR, CF1
    calculate the Precision of every class by: TP/TP+FP i.e. TP/total predict
    calculate the Recall by: TP/total GT
    """
    num, num_class = scores.shape
    gt_num = np.zeros(num_class)
    tp_num = np.zeros(num_class)
    predict_num = np.zeros(num_class)


    for index in range(num_class):
        score = scores[:, index]
        target = targets[:, index]

        gt_num[index] = np.sum(target == 1)
        predict_num[index] = np.sum(score >= 0.5)
        tp_num[index] = np.sum(target * (score >= 0.5))

    predict_num[predict_num == 0] = 1  # avoid dividing 0
    OP = np.sum(tp_num) / np.sum(predict_num) #OP (Overall Precision) is the ratio of the number of correctly predicted positive samples to the total number of positive predictions made by the model
    OR = np.sum(tp_num) / np.sum(gt_num) #OR (Overall Recall) is the ratio of the number of correctly predicted positive samples to the total number of positive samples in the ground truth.
    OF1 = (2 * OP * OR) / (OP + OR) #OF1 (Overall F1 Score) is the harmonic mean of precision and recall.

    return OP, OR, OF1

# Model 1: Resnet50

In [22]:
# Resnet50 model
from torchvision import models

class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Linear(2048, len(classes))

    def forward(self, x):
        x = self.resnet(x)
        return x

In [23]:
model = ResNet()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
from statistics import mean
import numpy as np

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model(data)

        preds = torch.round(output)
            
        #acc_list = []
        #preds = torch.round(output)
        #for i in range(len(preds)):
        #    result = 0
        #    denom = 0
        #    for j in range(len(classes)):
        #        if target[i][j] == 1 or preds[i][j] == 1:
        #            denom += 1
        #            if preds[i][j] == target[i][j]:
        #                result+=1
        #    acc_list.append(result/denom)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model(data)
        
        preds = torch.round(output)
        
        #acc_list = []
        #preds = torch.round(output)
        #for n in range(len(preds)):
        #    result = 0
        #    denom = 0
        #    for m in range(len(classes)):
        #        if target[n][m] == 1 or preds[n][m] == 1:
        #            denom += 1
        #            if preds[n][m] == target[n][m]:
        #                result+=1
        #    acc_list.append(result/denom)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)

    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP, OR, OF1))

for epoch in range(0, epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/3768 (0%)]	Loss: 0.705331 	OP: 0.516129	OR: 0.310680	OF1: 0.387879
Train Epoch: 0 [64/3768 (2%)]	Loss: 0.671548 	OP: 0.627451	OR: 0.313725	OF1: 0.418301
Train Epoch: 0 [128/3768 (3%)]	Loss: 0.665837 	OP: 0.560976	OR: 0.242105	OF1: 0.338235
Train Epoch: 0 [192/3768 (5%)]	Loss: 0.628026 	OP: 0.666667	OR: 0.233010	OF1: 0.345324
Train Epoch: 0 [256/3768 (7%)]	Loss: 0.612536 	OP: 0.648649	OR: 0.247423	OF1: 0.358209
Train Epoch: 0 [320/3768 (8%)]	Loss: 0.603931 	OP: 0.558824	OR: 0.211111	OF1: 0.306452
Train Epoch: 0 [384/3768 (10%)]	Loss: 0.592680 	OP: 0.620690	OR: 0.173077	OF1: 0.270677
Train Epoch: 0 [448/3768 (12%)]	Loss: 0.547110 	OP: 0.700000	OR: 0.214286	OF1: 0.328125
Train Epoch: 0 [512/3768 (14%)]	Loss: 0.553903 	OP: 0.724138	OR: 0.201923	OF1: 0.315789
Train Epoch: 0 [576/3768 (15%)]	Loss: 0.526018 	OP: 0.787879	OR: 0.265306	OF1: 0.396947
Train Epoch: 0 [640/3768 (17%)]	Loss: 0.523357 	OP: 0.718750	OR: 0.221154	OF1: 0.338235
Train Epoch: 0 [704/3768 (19%)]	Loss: 0.5

<ipython-input-24-ea5e5cd9fa3b>:45: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 0.4894 
OP: 0.416667
OR: 0.250000
OF1: 0.312500

Train Epoch: 1 [0/3768 (0%)]	Loss: 0.363536 	OP: 0.850000	OR: 0.515152	OF1: 0.641509
Train Epoch: 1 [64/3768 (2%)]	Loss: 0.354109 	OP: 0.887097	OR: 0.533981	OF1: 0.666667
Train Epoch: 1 [128/3768 (3%)]	Loss: 0.349617 	OP: 0.928571	OR: 0.559140	OF1: 0.697987
Train Epoch: 1 [192/3768 (5%)]	Loss: 0.375107 	OP: 0.819672	OR: 0.490196	OF1: 0.613497
Train Epoch: 1 [256/3768 (7%)]	Loss: 0.390494 	OP: 0.847458	OR: 0.505051	OF1: 0.632911
Train Epoch: 1 [320/3768 (8%)]	Loss: 0.347067 	OP: 0.842105	OR: 0.521739	OF1: 0.644295
Train Epoch: 1 [384/3768 (10%)]	Loss: 0.373509 	OP: 0.888889	OR: 0.554455	OF1: 0.682927
Train Epoch: 1 [448/3768 (12%)]	Loss: 0.321519 	OP: 0.903226	OR: 0.543689	OF1: 0.678788
Train Epoch: 1 [512/3768 (14%)]	Loss: 0.363520 	OP: 0.896552	OR: 0.500000	OF1: 0.641975
Train Epoch: 1 [576/3768 (15%)]	Loss: 0.357996 	OP: 0.900000	OR: 0.519231	OF1: 0.658537
Train Epoch: 1 [640/3768 (17%)]	Loss: 0.386962 	OP:

# Model 2: DenseNet201

In [25]:
import torchvision
import torch.nn as nn

model2 = torchvision.models.densenet201(pretrained=True)
num_ftrs = model2.classifier.in_features
model2.classifier = nn.Linear(num_ftrs, len(classes))

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model2.parameters(), lr=learning_rate)

/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [26]:
from statistics import mean
import numpy as np

def train(epoch):
    model2.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model2(data)

        preds = torch.round(output)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model2.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model2(data)
        
        preds = torch.round(output)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)

    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP, OR, OF1))

for epoch in range(0, epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/3768 (0%)]	Loss: 0.753938 	OP: 0.152174	OR: 0.074468	OF1: 0.100000
Train Epoch: 0 [64/3768 (2%)]	Loss: 0.716554 	OP: 0.387755	OR: 0.188119	OF1: 0.253333
Train Epoch: 0 [128/3768 (3%)]	Loss: 0.711073 	OP: 0.257143	OR: 0.098901	OF1: 0.142857
Train Epoch: 0 [192/3768 (5%)]	Loss: 0.690066 	OP: 0.437500	OR: 0.145833	OF1: 0.218750
Train Epoch: 0 [256/3768 (7%)]	Loss: 0.686749 	OP: 0.481481	OR: 0.121495	OF1: 0.194030
Train Epoch: 0 [320/3768 (8%)]	Loss: 0.670299 	OP: 0.448276	OR: 0.136842	OF1: 0.209677
Train Epoch: 0 [384/3768 (10%)]	Loss: 0.643616 	OP: 0.642857	OR: 0.180000	OF1: 0.281250
Train Epoch: 0 [448/3768 (12%)]	Loss: 0.615785 	OP: 0.685714	OR: 0.220183	OF1: 0.333333
Train Epoch: 0 [512/3768 (14%)]	Loss: 0.615868 	OP: 0.694444	OR: 0.245098	OF1: 0.362319
Train Epoch: 0 [576/3768 (15%)]	Loss: 0.608328 	OP: 0.709677	OR: 0.209524	OF1: 0.323529
Train Epoch: 0 [640/3768 (17%)]	Loss: 0.602450 	OP: 0.775000	OR: 0.295238	OF1: 0.427586
Train Epoch: 0 [704/3768 (19%)]	Loss: 0.6

<ipython-input-26-f4480feef50c>:33: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 0.4776 
OP: 0.642857
OR: 0.450000
OF1: 0.529412

Train Epoch: 1 [0/3768 (0%)]	Loss: 0.326022 	OP: 0.877193	OR: 0.555556	OF1: 0.680272
Train Epoch: 1 [64/3768 (2%)]	Loss: 0.352738 	OP: 0.946429	OR: 0.546392	OF1: 0.692810
Train Epoch: 1 [128/3768 (3%)]	Loss: 0.336630 	OP: 0.912281	OR: 0.520000	OF1: 0.662420
Train Epoch: 1 [192/3768 (5%)]	Loss: 0.372478 	OP: 0.882353	OR: 0.424528	OF1: 0.573248
Train Epoch: 1 [256/3768 (7%)]	Loss: 0.350243 	OP: 0.925926	OR: 0.510204	OF1: 0.657895
Train Epoch: 1 [320/3768 (8%)]	Loss: 0.341101 	OP: 0.849057	OR: 0.463918	OF1: 0.600000
Train Epoch: 1 [384/3768 (10%)]	Loss: 0.388773 	OP: 0.807018	OR: 0.442308	OF1: 0.571429
Train Epoch: 1 [448/3768 (12%)]	Loss: 0.332332 	OP: 0.909091	OR: 0.526316	OF1: 0.666667
Train Epoch: 1 [512/3768 (14%)]	Loss: 0.383255 	OP: 0.836364	OR: 0.489362	OF1: 0.617450
Train Epoch: 1 [576/3768 (15%)]	Loss: 0.362834 	OP: 0.875000	OR: 0.475728	OF1: 0.616352
Train Epoch: 1 [640/3768 (17%)]	Loss: 0.358341 	OP:

# Model 3: ResNet18

In [39]:
from torchvision import models

class ResNet152(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet152, self).__init__()
        self.resnet = models.resnet152(pretrained=True)
        self.resnet.fc = nn.Linear(2048, len(classes))

    def forward(self, x):
        x = self.resnet(x)
        return x

In [40]:
model3 = ResNet152()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model3.parameters(), lr=learning_rate)

In [41]:
from statistics import mean
import numpy as np

def train(epoch):
    model3.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model3(data)

        preds = torch.round(output)
            
        #acc_list = []
        #preds = torch.round(output)
        #for i in range(len(preds)):
        #    result = 0
        #    denom = 0
        #    for j in range(len(classes)):
        #        if target[i][j] == 1 or preds[i][j] == 1:
        #            denom += 1
        #            if preds[i][j] == target[i][j]:
        #                result+=1
        #    acc_list.append(result/denom)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model3.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model3(data)
        
        preds = torch.round(output)
        
        #acc_list = []
        #preds = torch.round(output)
        #for n in range(len(preds)):
        #    result = 0
        #    denom = 0
        #    for m in range(len(classes)):
        #        if target[n][m] == 1 or preds[n][m] == 1:
        #            denom += 1
        #            if preds[n][m] == target[n][m]:
        #                result+=1
        #    acc_list.append(result/denom)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)

    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP, OR, OF1))

for epoch in range(0, epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/3768 (0%)]	Loss: 0.700978 	OP: 0.142857	OR: 0.008696	OF1: 0.016393
Train Epoch: 0 [64/3768 (2%)]	Loss: 0.660682 	OP: 0.111111	OR: 0.009709	OF1: 0.017857
Train Epoch: 0 [128/3768 (3%)]	Loss: 0.623923 	OP: 0.200000	OR: 0.010526	OF1: 0.020000


<ipython-input-21-447663f93c68>:28: RuntimeWarning: invalid value encountered in double_scalars
  OF1 = (2 * OP * OR) / (OP + OR) #OF1 (Overall F1 Score) is the harmonic mean of precision and recall.


Train Epoch: 0 [192/3768 (5%)]	Loss: 0.628586 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [256/3768 (7%)]	Loss: 0.586905 	OP: 0.375000	OR: 0.032967	OF1: 0.060606
Train Epoch: 0 [320/3768 (8%)]	Loss: 0.577861 	OP: 0.400000	OR: 0.041237	OF1: 0.074766
Train Epoch: 0 [384/3768 (10%)]	Loss: 0.545859 	OP: 0.500000	OR: 0.074468	OF1: 0.129630
Train Epoch: 0 [448/3768 (12%)]	Loss: 0.558872 	OP: 0.541667	OR: 0.128713	OF1: 0.208000
Train Epoch: 0 [512/3768 (14%)]	Loss: 0.554358 	OP: 0.769231	OR: 0.192308	OF1: 0.307692
Train Epoch: 0 [576/3768 (15%)]	Loss: 0.524342 	OP: 0.657143	OR: 0.234694	OF1: 0.345865
Train Epoch: 0 [640/3768 (17%)]	Loss: 0.512286 	OP: 0.725000	OR: 0.295918	OF1: 0.420290
Train Epoch: 0 [704/3768 (19%)]	Loss: 0.544120 	OP: 0.577778	OR: 0.252427	OF1: 0.351351
Train Epoch: 0 [768/3768 (20%)]	Loss: 0.476676 	OP: 0.750000	OR: 0.354839	OF1: 0.481752
Train Epoch: 0 [832/3768 (22%)]	Loss: 0.529232 	OP: 0.692308	OR: 0.349515	OF1: 0.464516
Train Epoch: 0 [896/3768 (24%)]	Loss: 0.

<ipython-input-41-7b3794fc5e3a>:45: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 0.4822 
OP: 0.631579
OR: 0.600000
OF1: 0.615385

Train Epoch: 1 [0/3768 (0%)]	Loss: 0.340429 	OP: 0.928571	OR: 0.559140	OF1: 0.697987
Train Epoch: 1 [64/3768 (2%)]	Loss: 0.329169 	OP: 0.928571	OR: 0.547368	OF1: 0.688742
Train Epoch: 1 [128/3768 (3%)]	Loss: 0.298589 	OP: 0.953125	OR: 0.635417	OF1: 0.762500
Train Epoch: 1 [192/3768 (5%)]	Loss: 0.361942 	OP: 0.932203	OR: 0.523810	OF1: 0.670732
Train Epoch: 1 [256/3768 (7%)]	Loss: 0.373567 	OP: 0.885246	OR: 0.545455	OF1: 0.675000
Train Epoch: 1 [320/3768 (8%)]	Loss: 0.326641 	OP: 0.894737	OR: 0.525773	OF1: 0.662338
Train Epoch: 1 [384/3768 (10%)]	Loss: 0.322078 	OP: 0.875000	OR: 0.577320	OF1: 0.695652
Train Epoch: 1 [448/3768 (12%)]	Loss: 0.346426 	OP: 0.893939	OR: 0.572816	OF1: 0.698225
Train Epoch: 1 [512/3768 (14%)]	Loss: 0.327365 	OP: 0.925373	OR: 0.596154	OF1: 0.725146
Train Epoch: 1 [576/3768 (15%)]	Loss: 0.335234 	OP: 0.904762	OR: 0.564356	OF1: 0.695122
Train Epoch: 1 [640/3768 (17%)]	Loss: 0.347478 	OP: